In [1]:
import pandas as pd
import numpy as np
import time
import logging
import keras
import os
import random
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, QuantileTransformer, MinMaxScaler, Normalizer, RobustScaler, LabelEncoder
from tqdm import tqdm
from typing import List, Tuple, Generator, Iterator
np.random.seed(0)
%matplotlib inline
pd.set_option('display.max_columns', None)

###### **Preprocessing Data**

In [2]:
df = pd.read_csv('../Dataset/custom_data_sub100k.csv')
df

,flow_duration,flow_byte,src_port,dst_port,Duration,Rate,Srate,Drate,fin_flag_number,syn_flag_number,rst_flag_number,psh_flag_number,ack_flag_number,urg_flag_number,ece_flag_number,cwr_flag_number,ack_count,syn_count,fin_count,urg_count,rst_count,CoAP,HTTP,HTTPS,DNS,Telnet,SMTP,SSH,IRC,TCP,UDP,DHCP,ARP,ICMP,IGMP,IPv,LLC,Tot sum,Min,Max,AVG,Std,Tot size,IAT,Number,Magnitue,Radius,Covariance,Variance,Weight,DS status,Fragments,Sequence number,flow_idle_time,flow_active_time,Label
0,0.000193,832,80,61786,64,10369.107540,10369.107540,0,0,0,0,0,1,0,0,0,0,0,0,1,2,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,1,1501,54,778,187.625000,234.260505,54,0.000193,8,19.371371,321.382074,-14259.875000,11.639548,16,0,0,0,1.928810e-04,0.000193,0_normal
1,1.298898,7558,443,35442,55,11.548251,11.548251,0,0,0,0,0,1,0,0,0,0,2,0,4,14,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,1,4151,54,1506,319.307692,502.889709,1506,0.582391,13,25.182335,710.282604,-190.050000,1.019383,40,0,0,0,8.030751e-01,1.298898,0_normal
2,1.308947,21437,443,35442,55,22.155212,22.155212,0,0,0,0,0,1,0,0,0,0,2,0,7,28,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,1,5942,54,1506,848.857143,692.984318,1506,0.000132,7,38.716921,105.655099,0.000000,0.000000,12,0,0,0,1.320840e-04,1.308947,0_normal
3,1.427035,1477,35441,80,128,5.606030,5.606030,0,1,0,0,0,1,0,0,0,1,2,0,2,7,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,1,6391,54,1506,532.583333,649.853376,54,0.001591,12,35.448554,493.924529,0.000000,0.000000,35,0,0,0,1.267583e+00,1.427035,0_normal
4,1.480285,1585,35441,80,128,6.755456,6.755456,0,0,0,0,0,1,0,0,0,2,2,0,2,9,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,1,6553,54,1506,436.866667,611.959352,54,0.000684,15,30.470126,665.957497,0.000000,0.000000,56,0,0,0,7.359980e-04,1.480285,0_normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115067,0.002669,280,44216,80,64,1498.634748,1498.634748,0,0,0,1,0,1,0,0,0,0,2,1,0,3,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,1,66,66,66,66.000000,0.000000,66,0.000071,1,8.124038,0.000000,0.000000,0.000000,0,0,0,0,7.100000e-05,0.002669,TCP
115068,0.000000,86,53608,53,64,1498.634748,1498.634748,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,1,152,66,86,76.000000,10.000000,86,0.002155,2,8.717798,10.000000,0.000000,0.000000,0,0,0,0,1.716628e+09,0.000000,UDP
115069,0.000000,86,40828,53,64,43.190489,43.190489,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,1,1460,60,436,146.000000,144.706600,86,0.000656,10,15.099669,150.421630,0.000000,0.000000,9,0,0,0,1.716628e+09,0.000000,UDP
115070,0.001351,280,44246,80,64,2961.033533,2961.033533,0,0,0,1,0,1,0,0,0,0,2,1,0,3,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,1,2192,60,436,199.272727,155.298007,66,0.000100,11,16.905620,157.489555,0.000000,0.000000,10,0,0,0,9.990000e-05,0.001351,TCP


# INNR

### INNR function

In [4]:
from sklearn.metrics.pairwise import pairwise_kernels
from sklearn.metrics import classification_report

def K(X,Y=None,metric='poly',coef0=1,gamma=None,degree=3):
    if metric == 'poly':
        k = pairwise_kernels(X,Y=Y,metric=metric,coef0=coef0,gamma=gamma,degree=degree)
    elif metric == 'linear':
        k = pairwise_kernels(X,Y=Y,metric=metric)
    elif metric == 'sigmoid':
        k = pairwise_kernels(X,Y=Y,metric=metric,coef0=coef0,gamma=gamma)
    elif metric == 'rbf':
        k = pairwise_kernels(X,Y=Y,metric=metric,gamma=gamma)
    return k

def kernel_distance_matrix(matrix1 = None, matrix2 = None, kernel=None, gamma=None):
    """
    Calculate the distance between two matrices using the kernel trick.
    Parameters:
    - matrix1: The first input matrix (NumPy array).
    - matrix2: The second input matrix (NumPy array).
    - gamma: The gamma parameter for the RBF kernel.
    Returns:
    - distance_matrix: The distance matrix between the two input matrices.
    """

    if matrix1.shape[1] != matrix2.shape[1]:
        raise ValueError("The number of features in the input matrices must be the same.")
    Kaa = []
    for i in range(len(matrix1)):
        Kaa.append(K(matrix1[i,:].reshape(1,-1),metric=kernel))
    Kaa = np.asarray(Kaa).ravel().reshape(len(Kaa),1)
    Kab = K(matrix1,matrix2,metric=kernel)
    Kbb = []
    for i in range(len(matrix2)):
        Kbb.append(K(matrix2[i,:].reshape(1,-1),metric=kernel))
    Kbb = np.asarray(Kbb).ravel()
    d = Kaa-2*Kab+Kbb #shape: (matrix1,matrix2)
    return d

def calculate_accuracy_for_label(y_true, y_predict, label):
    """
    Calculate accuracy for a specific label.

    Parameters:
    - y_true: The true labels (1D NumPy array).
    - y_predict: The predicted labels (1D NumPy array).
    - label: The specific label for which to calculate accuracy.

    Returns:
    - accuracy: The accuracy for the specified label.
    """
    # Create a boolean mask for the specified label
    mask = (y_true == label)

    # Extract true labels and predicted labels for the specified label
    true_labels_for_label = y_true[mask]
    predicted_labels_for_label = y_predict[mask]


    # Calculate accuracy for the specified label
    accuracy = np.mean(true_labels_for_label == predicted_labels_for_label)
    # print(accuracy)
    return accuracy


class kINN:
    def __init__(self, R=1, kernel="linear", mode = "Supervise"):
        self.R = R
        self.kernel = kernel
        self.distance_matrix = None
        self.cluster_labels = None
        self.cluster_map = None
        self.N = None
        # self.n_clusters = n_clusters
        self.X = None
        self.M = None
        self.is_fit = False
        self.DNN_test = None
        self.distance_matrix_test = None
        self.mode = mode



    def _Bruteforce_threshold(self, y_test, y_pred, scores):
        # d = Decimal(np.min(scores))
        # min_th = max(1e-8,pow(10, d.as_tuple().exponent))
        # d = Decimal(np.max(scores))
        # max_th = min(1e-2,pow(10, d.as_tuple().exponent))

        # print("======================= DEBUG =======================")
        # print(min_th, max_th)
        # print("======================= DEBUG =======================")
        min_th = 1e-7
        max_th = 1e-1
        # __step = int(max_th / min_th)
        __step = 10000
        mcc = matthews_corrcoef(y_test, y_pred)
        mcc = 0
        ndr = 0
        y_pred_adv = y_pred


        __rag = np.unique(y_test)
        thr = -np.ones(len(__rag))

        for id in __rag[1:]:
            # print("DEBUG:", id)
            for x in np.linspace(min_th, max_th, num=__step):
                y_pred_adv_tmp = np.array([-1 if (y_p == id) and (sc > x) else y_p for y_p, sc in zip( y_pred,scores)])
                mcc_tmp = matthews_corrcoef(y_test, y_pred_adv_tmp)
                ndr_tmp = calculate_accuracy_for_label(y_test, y_pred_adv_tmp, -1)
                if np.mean([mcc_tmp*2,ndr_tmp]) > np.mean([mcc*2,ndr]):
                    y_pred_adv = y_pred_adv_tmp
                    mcc = mcc_tmp
                    ndr = ndr_tmp
                    thr[id] = x
            y_pred = y_pred_adv
        print("DEBUG - update mcc:", mcc, ndr, thr)
        return y_pred_adv, mcc, thr


    def _Bruteforce_threshold_1_cls(self, y_test, y_pred, scores):
          # d = Decimal(np.min(scores))
          # min_th = max(1e-8,pow(10, d.as_tuple().exponent))
          # d = Decimal(np.max(scores))
          # max_th = min(1e-2,pow(10, d.as_tuple().exponent))

          # print("======================= DEBUG =======================")
          # print(min_th, max_th)
          # print("======================= DEBUG =======================")
          min_th = 1e-7
          max_th = 1e-1
          # __step = int(max_th / min_th)
          __step = 10000
          mcc = matthews_corrcoef(y_test, y_pred)
          mcc = 0
          ndr = 0
          y_pred_adv = y_pred


          __rag = np.unique(y_test)
          thr = -np.ones(len(__rag))


          # for id in __rag[0]:
          id = 0
          for x in np.linspace(min_th, max_th, num=__step):
              y_pred_adv_tmp = np.array([1 if (y_p == id) and (sc > x) else y_p for y_p, sc in zip( y_pred,scores)])
              mcc_tmp = matthews_corrcoef(y_test, y_pred_adv_tmp)
              ndr_tmp = calculate_accuracy_for_label(y_test, y_pred_adv_tmp, 1)
              if np.mean([mcc_tmp*2,ndr_tmp]) > np.mean([mcc*2,ndr]):
                  y_pred_adv = y_pred_adv_tmp
                  mcc = mcc_tmp
                  ndr = ndr_tmp
                  thr[id] = x
          y_pred = y_pred_adv


          print("DEBUG - update mcc:", mcc, ndr, thr)
          return y_pred_adv, mcc, thr



    def fit(self, X, y = None, single = False, type = "distance"):
        """
        Fit the kINN model to the input data.

        Parameters:
        - X: Input data, a 2D numpy array where each row represents a sample.
        - y: Input label, a array where each row represents a label for corresponding label
        - type: the strategy to calculate distance, support:
                                                            + "distance" - use only distance
                                                            + "density" - use LOF score as weight when calculate distance
        """
        self.X = X
        if (y is None) and (self.mode == "1_Cls"):
          # -np.ones(self.N, dtype=int)
          y = np.zeros(X.shape[0], dtype=int)
        self._fit_classify(X,y,type)
        self.is_fit = True
        if single == True:
            X_new = self.__map_to_single_point(X)

            return X_new, self.cluster_labels, self.cluster_map
        else:
            return self.cluster_labels, self.cluster_map


    def _calculate_lof(self, dis_mat, k_dis, d_nn, N_cnt):
        # calculate reachability distance

        re_dis_k = np.array([ [ max(dis_mat[i,k],k_dis[k]) for k in range(N_cnt) ] for i in range(N_cnt)])

        # calculate Local Reachability Density (LRD)

        lrd = np.array([1. / np.mean(re_dis_k[x, d_nn[x]]) for x in range(N_cnt)])

        # calculate LOF
        lof = np.array([np.mean(lrd[d_nn[x]]) / lrd[x] for x in range(N_cnt)])

        return lof

    def _fit_classify(self, X, y, type="distance"):
        N = X.shape[0]
    
        D_NN = np.empty((N,), dtype=object)
        INNR = np.empty((N,), dtype=object)
    
        classes, cls_cnt = np.unique(y, return_counts=True)
    
        for cls, N_cnt in zip(classes, cls_cnt):
            indicates = np.asarray(y == cls).nonzero()[0]
            X_cls = X[indicates]
            y_cls = y[indicates]
    
            dis_mat = kernel_distance_matrix(matrix1=X_cls, matrix2=X_cls, kernel=self.kernel)
    
            if type == "density":
                k_dis = -np.ones(N_cnt, dtype=float)
                d_nn = np.empty((N_cnt,), dtype=object)
                re_dis_k = -np.ones((N_cnt, N_cnt), dtype=float)
    
                for i in range(N_cnt):
                    dis_mat[i, i] = 0
                    tmp = dis_mat[i, :].argsort()
    
                    # Some case that 2 point are too close
                    dnn_tmp = [i]
                    cnt = 0
                    for x in tmp:
                        if abs(dis_mat[i, dnn_tmp[-1]] - dis_mat[i, x]) > 1e-9:
                            dnn_tmp.append(x)
                            cnt += 1
                            k_dis[i] = dis_mat[i, x]
                        else:
                            dnn_tmp.append(x)
    
                        if cnt >= self.R:
                            break
                    d_nn[i] = np.array(dnn_tmp[1:])
    
                lof = self._calculate_lof(dis_mat, k_dis, d_nn, N_cnt)
    
                # Update new matrix
                dis_mat = np.array([[dis_mat[i, k] * lof[k] for k in range(N_cnt)] for i in range(N_cnt)])
    
            # calculate D_N
            for i in range(N_cnt):
                dis_mat[i, i] = 0
                tmp = dis_mat[i, :].argsort()
    
                id = indicates[i]
                D_NN[id] = np.array(indicates[tmp])
    
                # Some case that 2 point are too close
                dnn_tmp = [i]
                cnt = 0
                for x in tmp:
                    if abs(dis_mat[i, dnn_tmp[-1]] - dis_mat[i, x]) > 1e-9:
                        dnn_tmp.append(x)
                        cnt += 1
                    else:
                        dnn_tmp.append(x)
    
                    if cnt >= self.R:
                        break
    
                D_NN[id] = np.array(indicates[dnn_tmp[1:]])
    
            self.D_NN = D_NN
    
            for i in tqdm(indicates, desc=f"Processing class {cls}"):
                NN = D_NN[i]
                tmp = []
                for p in NN:
                    p_near_neighbor = D_NN[p]
                    if i in p_near_neighbor:
                        tmp.append(p)
    
                pair = (i, tmp)
                INNR[i] = pair
    
        self.INNR = INNR
        self.N = N
        self.cluster_labels, self.no_cluser = self._label_clusters()
    
        cluster_map = np.full(self.no_cluser, -1)
    
        for x_tmp, y_tmp in zip(self.cluster_labels, y):
            if cluster_map[x_tmp] == -1:
                cluster_map[x_tmp] = y_tmp
            else:
                if cluster_map[x_tmp] != y_tmp:
                    print("Debug - Loi KNN")
    
        self.cluster_map = cluster_map


    # def _fit_cluster(self,X):
    #     N = X.shape[0]
    #     dis_mat = kernel_distance_matrix(matrix1 = X, matrix2 = X, kernel = self.kernel)
    #     D_NN = []
    #     for i in range(N):
    #         dis_mat[i,i] = 0
    #         tmp = dis_mat[i,].argsort()
    #         D_NN.append(tmp)

    #     D_NN = np.array(D_NN)
    #     self.D_NN = D_NN
    #     INNR = []

    #     for i in range(N):
    #         NN = D_NN[i, 1:self.R+1]

        #     tmp = []
        #     for p in NN:
        #         p_near_neighbor = D_NN[p, 1:self.R+1]
        #         if i in p_near_neighbor:
        #             tmp.append(p)

        #     pair = (i, tmp)
        #     INNR.append(pair)
        # self.INNR = INNR

        # self.N = N
        # self.cluster_labels, self.no_cluser = self._label_clusters()

    def _label_clusters(self):
        """
        Label clusters using deep find search (DFS) algorithm.

        Parameters:
        - X: Input data, a 2D numpy array where each row represents a sample.

        Returns:
        - labels: A numpy array containing cluster labels for each sample.

        """
        # print(self.INN)
        labels = -np.ones(self.N, dtype=int)
        current_label = 0

        # INNR = sorted(self.INNR,key=lambda x: len(x[1]), reverse=True)
        for x in self.INNR:
            id = x[0]
            if labels[id] == -1:
                queue = [id]
                labels[id] = current_label
                # _debug = 0
                for q in queue:
                    neighbors = self.INNR[q][1]
                    for neighbor in neighbors:
                        if labels[neighbor] == -1:
                            queue.append(neighbor)
                            labels[neighbor] = current_label
                current_label += 1
        return labels, current_label

    def _dfs_label_clusters(self, i, current_label, labels):
        """
        DFS algorithm to label clusters.

        Parameters:
        - X: Input data, a 2D numpy array where each row represents a sample.
        - i: Index of the current sample being explored.
        - current_label: Current cluster label.
        - labels: A numpy array containing cluster labels for each sample.

        """
        if labels[i] != -1:
            return

        labels[i] = current_label

        neighbors = self.INNR[i][1]
        for neighbor in neighbors:
            #
            self._dfs_label_clusters(neighbor, current_label, labels)

    def __map_to_single_point(self, X):

        a = self.cluster_labels
        x_new = []
        a_new = []
        for cl in np.unique(a):
            mask = np.isin(a, cl)
            known = X[mask]
            a_new.append(self.cluster_map[cl])
        # print(x_new)
        # print(a_new)
        self.cluster_labels = np.array(a_new)


        return np.array(x_new)

    def find_nearest_neighbors(self, X, x_i):
        """
        Find the R nearest neighbors of x_i using kernel trick.

        Parameters:
        - x_i: The input sample.

        Returns:
        - neighbors: A set of R nearest neighbors for x_i.

        """
        if self.distance_matrix is None:
            raise ValueError("Model not fitted. Call fit() first.")

        # Calculate kernel vector
        kernel_vector = self.distance_matrix[x_i].flatten()

        # Get indices of R nearest neighbors
        nearest_indices = np.argsort(kernel_vector)[1:self.R+1]

        return set(nearest_indices)


    def predict(self, X_test, y = None):
        """
        Predict the cluster labels for the input samples.

        Parameters:
        - X_test: Input data, a 2D numpy array where each row represents a sample.

        Returns:
        - predicted_labels: A numpy array containing predicted cluster labels for each sample.

        """
        if self.is_fit == False:
            raise ValueError("Model not fitted. Call fit() first.")


        N_test = X_test.shape[0]
        self.M = N_test
        dis_mat_X_test = kernel_distance_matrix(matrix1 = X_test, matrix2 = self.X, kernel = self.kernel)
        self.distance_matrix_test = dis_mat_X_test
        D_NN_test = []
        # print("Khoảng cách X_test -> X_train:\n ",dis_mat_X_test,"\n")
        for i in range(N_test):
            # dis_mat_X_test[i,i] = 0
            tmp = dis_mat_X_test[i,].argsort()
            D_NN_test.append(tmp)

        D_NN_test = np.array(D_NN_test)
        # print(D_NN_test.shape)
        self.D_NN_test = D_NN_test
        INNR_X_test = []
        # Tìm INNR_X_test
        for i in range(N_test):
            NN = D_NN_test[i, 1:self.R+1]
            # print(NN)
            tmp = []
            for p in NN:
                p_near_neighbor = D_NN_test.T[p, 1:self.R+1]
                # print("neighbor: ",p_near_neighbor)
                if i in p_near_neighbor:
                    # print(p_near_neighbor)
                    tmp.append(p)
            pair = (i, tmp)
            # print(pair)
            INNR_X_test.append(pair)
        self.INNR_test = INNR_X_test


        if self.mode == "Supervise":
            labels = self._predict_multi()
            return labels
        elif self.mode == "Novelty_multi":
            labels,mcc, threshold  = self._predict_novelty(y)
            return labels , mcc, threshold
        else:
            labels,mcc, threshold = self._predict_1Class(y)
            return labels, mcc, threshold



    def _predict_multi(self):
        labels = -np.ones(self.M, dtype=int)
        # c = 0
        for pair in self.INNR_test:
                idx = pair[0]
                if pair[1] != []:  # Kiểm tra xem danh sách neighbor có rỗng không
                    neighbors = pair[1]
                    # print(id, "neigibor = ",neighbors)
                    labels[idx] = self.cluster_map[self.cluster_labels[neighbors[0]]]
                    # c+= 1
                    # print(self.cluster_labels[neighbors[0]])
                else:
                    labels[idx] = self.cluster_map[self.cluster_labels[self.D_NN_test[idx][0]]]
        # print("Count: ",c)
        return labels


    def _predict_novelty(self, y):
        scores_mat = self.distance_matrix_test
        y_pred = [self.cluster_labels[x] for x in np.argmin(scores_mat, axis=1)]

        for i in range(self.M):
            y_pred[i] = self.cluster_map[y_pred[i]]

        scores = np.amin(scores_mat, axis=1)
        y_pred_adv, mcc, threshold = self._Bruteforce_threshold(y, y_pred, scores)


        print(f"MCC: {mcc}", f"threshold: {threshold}")

        return y_pred_adv, mcc, threshold


    def _predict_1Class(self,y):

        scores_mat = self.distance_matrix_test
        # print(scores_mat)
        # y_pred = [self.cluster_labels[x] for x in np.argmin(scores_mat, axis=1)]

        # for i in range(self.M):
        #     y_pred[i] = self.cluster_map[y_pred[i]]
        y_pred = np.zeros(self.M, dtype = int)
        scores = np.amin(scores_mat, axis=1)
        y_pred_adv, mcc, threshold = self._Bruteforce_threshold_1_cls(y, y_pred, scores)
        # print("DEBUG- y_pred: ", y_pred)

        print(f"MCC: {mcc}", f"threshold: {threshold}")

        return y_pred_adv, mcc, threshold

In [5]:
data_train, data_test, target_train, target_test = train_test_split(df, df["Label"], test_size=0.3, stratify = df["Label"], random_state=42)
#Chuyển dataframe --> mảng numpy
X_test_1 = data_test.to_numpy()
#Bỏ cột cuối (Label)
X_test = X_test_1[:,:-1]

In [6]:
### Parameters ----------------------------------------------------------------
__SEED = 42
__KNN = 25
__KERNEL = "rbf"
__TYPE = "density"
__TARGET = "Label"
__SCALER = "QuantileTransformer"
drop_cls = ["*"]
__MODE = "Supervise"

In [7]:
import pickle
# Load model và các thành phần từ file
with open("kinn_model.pkl", "rb") as f:
    saved_data = pickle.load(f)

# Truy xuất các thành phần
kinn_model_loaded = saved_data["model"]
cluster_train = saved_data["cluster_train"]
cluster_map = saved_data["cluster_map"]
parameters = saved_data["parameters"]

# In thông tin đã tải
print("Model đã được tải thành công!")
print("Thông số đã lưu:", parameters)

Model đã được tải thành công!
Thông số đã lưu: {'SEED': 42, 'KNN': 25, 'KERNEL': 'rbf', 'TYPE': 'density', 'TARGET': 'Label', 'SCALER': 'QuantileTransformer', 'MODE': 'Supervise'}


In [11]:
#Scaler bằng QuantileTransformer
scaler_loaded = joblib.load('scaler.pkl')
encoder_loaded = joblib.load('label_encoder.pkl')
X_test_scaled = scaler_loaded.transform(X_test)

In [12]:
X_test_scaled[1].shape

(55,)

In [13]:
y_pred = kinn_model_loaded.predict(X_test_scaled[:200])
y_pred

array([0, 3, 3, 3, 0, 3, 3, 0, 0, 3, 3, 3, 0, 3, 0, 3, 0, 0, 3, 0, 0, 0,
       3, 3, 3, 3, 3, 0, 0, 3, 3, 0, 0, 3, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0,
       3, 0, 0, 3, 3, 3, 0, 0, 3, 0, 0, 0, 0, 3, 3, 0, 3, 0, 3, 3, 3, 3,
       0, 0, 0, 3, 3, 0, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 0,
       0, 0, 3, 0, 0, 3, 3, 0, 3, 3, 0, 0, 0, 3, 0, 3, 0, 0, 0, 0, 0, 0,
       0, 3, 3, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 3, 3, 3, 3, 0, 3, 0,
       0, 3, 0, 0, 0, 0, 0, 3, 0, 3, 0, 0, 0, 0, 0, 3, 3, 0, 3, 0, 0, 0,
       0, 0, 0, 3, 3, 3, 0, 0, 3, 0, 3, 0, 0, 3, 0, 3, 0, 0, 3, 0, 0, 0,
       0, 3, 0, 0, 0, 3, 0, 0, 3, 0, 0, 0, 3, 3, 0, 3, 3, 3, 0, 0, 3, 0,
       3, 3])